# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
# import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
variaveis_qualitativas = df[['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']]

# for variavel in variaveis_qualitativas:

   #print(f"\nCategorias para '{variavel}':")

     #categorias = df[variavel].value_counts()

     #print(categorias)

     #print(f"\nCategorias de '{variavel}' Ordenadas:")

     #ordenadas = categorias.sort_values(ascending=False)

     #print(ordenadas) */

In [10]:
df['log_renda'] = np.log(df['renda'])

# Definindo a fórmula com variáveis qualitativas (usando a maior categoria como referência)
formula = 'log_renda ~ C(sexo, Treatment("F")) + C(tipo_renda, Treatment("Assalariado")) + C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia'
y, X = patsy.dmatrices(formula, data=df)

modelo = sm.OLS(y, X).fit()

print(modelo.summary())


                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     401.5
Date:                Sat, 21 Dec 2024   Prob (F-statistic):               0.00
Time:                        23:25:56   Log-Likelihood:                -13593.
No. Observations:               12427   AIC:                         2.722e+04
Df Residuals:                   12409   BIC:                         2.735e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

- Tirado o tipo_residencia por apresentar o p value menos signficante para o modelo.
- Após remoção, não houve alteração no r-squared (R-squared:0.355 e) o AIC apresentou uma melhora, assim como r-squared ajustado.

In [12]:
df['log_renda'] = np.log(df['renda'])

# Definindo a fórmula com variáveis qualitativas (usando a maior categoria como referência)
formula = 'log_renda ~ C(sexo, Treatment("F")) + C(estado_civil, Treatment("Casado")) + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia'
y, X = patsy.dmatrices(formula, data=df)

# Tirado o tipo_residencia por apresentar o p value menos signficante para o modelo (não houve alteração no r-squared).
modelo2 = sm.OLS(y, X).fit()

print(modelo2.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     724.1
Date:                Sat, 21 Dec 2024   Prob (F-statistic):               0.00
Time:                        23:31:30   Log-Likelihood:                -13695.
No. Observations:               12427   AIC:                         2.741e+04
Df Residuals:                   12417   BIC:                         2.748e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

- Modelo 1: É mais complexo, mas provavelmente oferece melhor capacidade preditiva, pois considera todas as variáveis relevantes, inclusive aquelas que poderiam ter um impacto pequeno, mas não insignificante.
- Modelo 2: É mais simples, o que o torna mais interpretável e menos sujeito a sobreajuste (overfitting), mas pode perder precisão preditiva porque ignora variáveis potencialmente úteis.


### Conclusão 

Modelo 1 é superior ao Modelo 2, considerando o AIC e o BIC, que equilibram ajuste e complexidade. Embora o Modelo 2 seja mais simples, ele sacrifica a qualidade do ajuste. Portanto, o Modelo 1 é preferível, especialmente se o objetivo for maximizar a explicação da variabilidade de log_renda.

Se o objetivo for simplicidade e interpretabilidade, o Modelo 2 poderia ser escolhido, mas isso depende do contexto e da finalidade da análise.
